<a href="https://colab.research.google.com/github/81300/AI-Notebooks/blob/testing/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM text generation notebook for Google Colab

Version: GPU

This notebook uses **oobabooga**'s [text-generation-webui project](https://github.com/oobabooga/text-generation-webui). Read [this page](https://github.com/oobabooga/text-generation-webui/blob/main/README.md) and [the wiki](https://github.com/oobabooga/text-generation-webui/wiki) to learn more about the available features. Check out his own updated Colab [here](https://colab.research.google.com/github/oobabooga/AI-Notebooks/blob/main/Colab-TextGen-GPU.ipynb).

For PygmalionAI models you can create your custom characters using the [JSON character creator](https://oobabooga.github.io/character-creator.html).

---

Run all the cells and a public gradio URL will appear at the bottom.

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

# From: https://github.com/henk717/KoboldAI
#@markdown Press play on the music player to keep the tab alive (uses only 13MB of data).
%%html
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title 2. Install the web UI

Enable_Google_Drive = False #@param {type:"boolean"}


import os
repo_runtime_path = "/content/Colab-TextGen/"
repo_drive_path = "/content/drive/MyDrive/Colab-TextGen/"
if Enable_Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')
  repo_path = repo_drive_path
else:
  repo_path = repo_runtime_path
if not os.path.exists(repo_path):
  os.mkdir(repo_path)


# The web UI
%cd $repo_path
![[ ! -d .git ]] && git clone https://github.com/oobabooga/text-generation-webui .
!git pull
!pip install --no-cache-dir --progress-bar=off --upgrade -r requirements.txt -r extensions/google_translate/requirements.txt
!wget -O settings-colab.json https://oobabooga.github.io/settings-colab.json
!cp -a settings-colab.json settings-colab-template.json

In [ ]:
#@title 3. Select a model

Model = "Pygmalion 6B original (sharded, rehosted)" #@param ["Pygmalion 6B main", "Pygmalion 6B main (sharded, rehosted)", "Pygmalion 6B original", "Pygmalion 6B original (sharded, rehosted)", "Pygmalion 6B dev", "Pygmalion 2.7B", "Pygmalion 1.3B", "Pygmalion 350M"] {allow-input: false}
Delete_all_models = False #@param {type: "boolean"}

models = {
    "Pygmalion 6B main": ("PygmalionAI", "pygmalion-6b", "main", "pygmalion-6b"),
    "Pygmalion 6B main (sharded, rehosted)": ("ayylamo", "pygmalion-6b", "sharded", "pygmalion-6b_sharded"),
    "Pygmalion 6B original": ("PygmalionAI", "pygmalion-6b", "b8344bb4eb76a437797ad3b19420a13922aaabe1", "pygmalion-6b_b8344bb4eb76a437797ad3b19420a13922aaabe1"),
    "Pygmalion 6B original (sharded, rehosted)": ("ayylamo", "pygmalion-6b", "original-sharded", "pygmalion-6b_original-sharded"),
    "Pygmalion 6B dev": ("PygmalionAI", "pygmalion-6b", "dev", "pygmalion-6b_dev"),
    "Pygmalion 2.7B": ("PygmalionAI", "pygmalion-2.7b", "main", "pygmalion-2.7b"),
    "Pygmalion 1.3B": ("PygmalionAI", "pygmalion-1.3b", "main", "pygmalion-1.3b"),
    "Pygmalion 350M": ("PygmalionAI", "pygmalion-350m", "main", "pygmalion-350m")
}
huggingface_org, huggingface_repo, huggingface_branch, model_name = models[Model]
models_path = repo_path + f"models"


if Delete_all_models:
  !find $models_path -mindepth 1 -maxdepth 1 -type d -exec rm -rv "{}" \;

%cd $repo_path
![[ ! -f models/$model_name/config.json ]] && python download-model.py $huggingface_org/$huggingface_repo --branch $huggingface_branch

In [ ]:
#@title 4. Launch

UI_mode = "Chat" #@param ["Chat", "Notebook"] {allow-input: false}
Chat_language = "English" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']
language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}


%cd $repo_path
if UI_mode == "Chat" and Chat_language == "English":
  !python server.py --share --model $model_name --settings settings-colab.json --cai-chat
elif UI_mode == "Chat" and Chat_language != "English":
  import json
  j = json.loads(open('settings-colab-template.json', 'r').read())
  j["google_translate-language string"] = language_codes[Chat_language]
  with open('settings-colab.json', 'w') as f:
    f.write(json.dumps(j, indent=4))
  !python server.py --share --model $model_name --settings settings-colab.json --cai-chat --extensions google_translate --no-stream
elif UI_mode == "Notebook":
  !python server.py --share --model $model_name --settings settings-colab.json --notebook
else:
  print("No valid mode selected!")